In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

from path_setup import setup_paths
setup_paths()

In [5]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [6]:
df = pd.read_csv('../datasets/NYC311data.csv')

C:\Users\Dmitriy.Burnashev\AppData\Local\Temp\ipykernel_33300\2014850805.py:1: DtypeWarning: Columns (48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../datasets/NYC311data.csv')


In [7]:
columns_to_delete = [
'School Phone Number',
'School Address',
'School City',
'School State',
'School Zip',
'School Not Found',
'School or Citywide Complaint',
'Vehicle Type',
'Taxi Company Borough',
'Taxi Pick Up Location',
'Bridge Highway Name',
'Bridge Highway Direction',
'Road Ramp',
'Bridge Highway Segment',
'Garage Lot Name',
'Ferry Direction',
'Ferry Terminal Name',

'Location',
'School Name',
'School Number',
'Intersection Street 1',
'Intersection Street 2',
'Landmark',
'Incident Address',
'Unique Key',
'Agency',
'Agency Name',
'Status',
'Due Date', 
'Resolution Description', 
'Resolution Action Updated Date',
'Closed Date',
'School Region',	
'School Code',
'Park Facility Name',
'X Coordinate (State Plane)',
'Y Coordinate (State Plane)'
]
df = df.drop(columns_to_delete,axis=1)

In [24]:
df['Incident Zip'] = df['Incident Zip'].astype('category')

In [9]:
def get_time_features(df,column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    
    df[f'hour_sin_{column_name}'] = np.sin(2 * np.pi * df[column_name].dt.hour / 24)
    df[f'hour_cos_{column_name}'] = np.cos(2 * np.pi * df[column_name].dt.hour / 24)

    df[f'weekday_sin_{column_name}'] = np.sin(2 * np.pi * df[column_name].dt.weekday / 7)
    df[f'weekday_cos_{column_name}'] = np.cos(2 * np.pi * df[column_name].dt.weekday / 7)

    df[f'month_{column_name}'] = df[column_name].dt.month

    df[f'day_{column_name}'] = df[column_name].dt.day

    df = df.drop(column_name,axis=1)
    return df

In [10]:
df = get_time_features(df,'Created Date')
df.head()

C:\Users\Dmitriy.Burnashev\AppData\Local\Temp\ipykernel_33300\4267985790.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column_name] = pd.to_datetime(df[column_name])


,Complaint Type,Descriptor,Location Type,Incident Zip,Street Name,Cross Street 1,Cross Street 2,Address Type,City,Facility Type,...,Borough,Park Borough,Latitude,Longitude,hour_sin_Created Date,hour_cos_Created Date,weekday_sin_Created Date,weekday_cos_Created Date,month_Created Date,day_Created Date
0,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10034.0,VERMILYEA AVENUE,ACADEMY STREET,WEST 204 STREET,ADDRESS,NEW YORK,Precinct,...,MANHATTAN,MANHATTAN,40.865682,-73.923501,-0.258819,0.965926,0.433884,-0.900969,12,31
1,Blocked Driveway,No Access,Street/Sidewalk,11105.0,23 AVENUE,27 STREET,28 STREET,ADDRESS,ASTORIA,Precinct,...,QUEENS,QUEENS,40.775945,-73.915094,-0.258819,0.965926,0.433884,-0.900969,12,31
2,Blocked Driveway,No Access,Street/Sidewalk,10458.0,VALENTINE AVENUE,EAST 198 STREET,EAST 199 STREET,ADDRESS,BRONX,Precinct,...,BRONX,BRONX,40.870325,-73.888525,-0.258819,0.965926,0.433884,-0.900969,12,31
3,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,10461.0,BAISLEY AVENUE,EDISON AVENUE,B STREET,ADDRESS,BRONX,Precinct,...,BRONX,BRONX,40.835994,-73.828379,-0.258819,0.965926,0.433884,-0.900969,12,31
4,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11373.0,57 ROAD,SEABURY STREET,HOFFMAN DRIVE,ADDRESS,ELMHURST,Precinct,...,QUEENS,QUEENS,40.733060,-73.874170,-0.258819,0.965926,0.433884,-0.900969,12,31


In [11]:
from sklearn.cluster import KMeans
coords = df[['Latitude', 'Longitude']].dropna()
kmeans = KMeans(n_clusters=10, random_state=42).fit(coords)
df['geo_cluster'] = kmeans.predict(df[['Latitude', 'Longitude']].fillna(0))

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
#center_lat = df['Latitude'].mean()
#center_lon = df['Longitude'].mean()
#
#from geopy.distance import geodesic
#
#df['distance_to_center'] = df.apply(lambda row: geodesic((row['Latitude'], row['Longitude']),
#                                                         (center_lat, center_lon)).kilometers, axis=1)

In [12]:

df['combined_text'] = df[['Descriptor', 'Location Type', 'Street Name', 'Cross Street 1', 'Cross Street 2', 
                          'City', 'Facility Type', 'Community Board', 'Borough', 'Park Borough']].fillna('').agg(', '.join, axis=1)


In [13]:
c = ['Descriptor', 'Location Type', 'Street Name', 'Cross Street 1', 'Cross Street 2', 
                          'City', 'Facility Type', 'Community Board', 'Borough', 'Park Borough','Latitude','Longitude']
df = df.drop(c,axis=1)

In [14]:
df.head()

,Complaint Type,Incident Zip,Address Type,hour_sin_Created Date,hour_cos_Created Date,weekday_sin_Created Date,weekday_cos_Created Date,month_Created Date,day_Created Date,geo_cluster,combined_text
0,Noise - Street/Sidewalk,10034.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,3,"Loud Music/Party, Street/Sidewalk, VERMILYEA A..."
1,Blocked Driveway,11105.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,7,"No Access, Street/Sidewalk, 23 AVENUE, 27 STRE..."
2,Blocked Driveway,10458.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,1,"No Access, Street/Sidewalk, VALENTINE AVENUE, ..."
3,Illegal Parking,10461.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,1,"Commercial Overnight Parking, Street/Sidewalk,..."
4,Illegal Parking,11373.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,7,"Blocked Sidewalk, Street/Sidewalk, 57 ROAD, SE..."


In [13]:
"""
import pandas as pd

wifi_points = []
for i in df.index:
    wifi_points.append({'index': i, 'Coordinates': [df['Latitude'][i], df['Longitude'][i]]})
    

from folium import Map, CircleMarker
m = Map(center=(df['Latitude'].mean(), df['Longitude'].mean()))

for i in range(len(wifi_points)):
    latitude, longitude = wifi_points[i]['Coordinates'] 
    
    point = CircleMarker(location=(latitude, longitude), radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=0.7)

    m.add_child(point)
m
"""

"\nimport pandas as pd\n\nwifi_points = []\nfor i in df.index:\n    wifi_points.append({'index': i, 'Coordinates': [df['Latitude'][i], df['Longitude'][i]]})\n    \n\nfrom folium import Map, CircleMarker\nm = Map(center=(df['Latitude'].mean(), df['Longitude'].mean()))\n\nfor i in range(len(wifi_points)):\n    latitude, longitude = wifi_points[i]['Coordinates'] \n    \n    point = CircleMarker(location=(latitude, longitude), radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=0.7)\n\n    m.add_child(point)\nm\n"

In [14]:
"""
def calculate_bearing(lat1, lon1, lat2, lon2):
    d_lon = np.radians(lon2 - lon1)
    lat1, lat2 = np.radians(lat1), np.radians(lat2)
    x = np.sin(d_lon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_lon)
    bearing = np.degrees(np.arctan2(x, y))
    return (bearing + 360) % 360


df['bearing_to_center'] = df.apply(lambda row: calculate_bearing(row['Latitude'], row['Longitude'], center_lat, center_lon), axis=1)
"""

"\ndef calculate_bearing(lat1, lon1, lat2, lon2):\n    d_lon = np.radians(lon2 - lon1)\n    lat1, lat2 = np.radians(lat1), np.radians(lat2)\n    x = np.sin(d_lon) * np.cos(lat2)\n    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_lon)\n    bearing = np.degrees(np.arctan2(x, y))\n    return (bearing + 360) % 360\n\n\ndf['bearing_to_center'] = df.apply(lambda row: calculate_bearing(row['Latitude'], row['Longitude'], center_lat, center_lon), axis=1)\n"

In [24]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np
from tqdm import tqdm

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

batch_size = 1024

def get_embeddings(df, batch_size):
    all_embeddings = []

    for i in tqdm(range(0, len(df), batch_size)):
        batch_texts = df['combined_text'].tolist()[i:i+batch_size]

        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
        
        with torch.no_grad():
            outputs = model(**encoded_input)
        
        cls_embeddings = outputs.last_hidden_state[:, 0, :].numpy() 
        all_embeddings.append(cls_embeddings)

    return np.vstack(all_embeddings)

cls_embeddings = get_embeddings(df, batch_size)




100%|██████████| 294/294 [1:40:51<00:00, 20.58s/it]


In [25]:
#np.save('cls_embeddings.npy', cls_embeddings)

In [16]:
cls_embeddings = np.load("cls_embeddings.npy")

In [17]:
cls_embeddings.shape[1]

768

In [18]:
len(cls_embeddings[0])

768

In [19]:
col = [f'{i+1}_feature' for i in range(cls_embeddings.shape[1])]

df_body = pd.DataFrame(cls_embeddings, columns=col)

df = pd.concat([df, df_body], axis=1)


In [20]:
df

,Complaint Type,Incident Zip,Address Type,hour_sin_Created Date,hour_cos_Created Date,weekday_sin_Created Date,weekday_cos_Created Date,month_Created Date,day_Created Date,geo_cluster,...,759_feature,760_feature,761_feature,762_feature,763_feature,764_feature,765_feature,766_feature,767_feature,768_feature
0,Noise - Street/Sidewalk,10034.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,3,...,0.243427,-0.278032,-0.093538,-0.302995,0.120089,-0.227137,-0.104675,-0.012241,0.035997,0.317733
1,Blocked Driveway,11105.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,7,...,0.177499,-0.337157,-0.139324,-0.370589,0.220497,-0.352660,-0.050506,-0.018762,-0.014149,0.647060
2,Blocked Driveway,10458.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,1,...,0.104849,-0.207178,-0.003043,-0.250431,0.330262,-0.331331,-0.047628,-0.088640,0.012561,0.719715
3,Illegal Parking,10461.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,1,...,0.121953,-0.303619,-0.073429,-0.244965,0.309406,-0.348047,-0.031682,-0.128702,0.039444,0.464534
4,Illegal Parking,11373.0,ADDRESS,-0.258819,0.965926,0.433884,-0.900969,12,31,7,...,0.263097,-0.163885,-0.199695,-0.256388,0.408909,-0.333840,-0.078009,0.008854,-0.003736,0.541290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300693,Noise - Commercial,NaN,NaN,0.000000,1.000000,-0.781831,0.623490,3,29,4,...,0.237798,-0.041400,0.042538,-0.135642,0.072827,-0.358408,-0.051388,-0.171698,0.195899,0.409011
300694,Blocked Driveway,11418.0,ADDRESS,0.000000,1.000000,-0.781831,0.623490,3,29,0,...,0.272000,-0.257961,-0.159445,-0.290121,0.344980,-0.299917,-0.101550,0.045796,-0.117795,0.653029
300695,Noise - Commercial,11206.0,ADDRESS,0.000000,1.000000,-0.781831,0.623490,3,29,6,...,0.278222,-0.470996,-0.181262,-0.336969,0.139486,-0.142126,-0.124531,-0.044669,0.062476,0.229135
300696,Noise - Commercial,10461.0,ADDRESS,0.000000,1.000000,-0.781831,0.623490,3,29,1,...,0.208949,-0.381882,-0.175328,-0.279591,0.090795,-0.191244,-0.066899,-0.171172,0.068957,0.283862


In [21]:
bdf = BaseDatasetTransform(df,target='Complaint Type')

In [22]:
df.isna().any()

Complaint Type           False
Incident Zip              True
Address Type              True
hour_sin_Created Date    False
hour_cos_Created Date    False
                         ...  
764_feature              False
765_feature              False
766_feature              False
767_feature              False
768_feature              False
Length: 779, dtype: bool

In [23]:
X_train, X_test, y_train, y_test,categorical_features = bdf.fit_transform()


Обнаружены колонки с именем "id": ['Incident Zip']
Series([], dtype: int64)

В колонке(ах) ['Complaint Type'] нет пропущенных значений
В наборе данных есть пропущенные значения:
Колонки с пропущенными значениями:
Incident Zip    1827
Address Type    1962
dtype: int64
Все пропущенные значения в обучающем наборе заполнены.
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Address Type', 'combined_text'], dtype='object')

Числовые колонки:
Index(['Incident Zip', 'hour_sin_Created Date', 'hour_cos_Created Date',
       'weekday_sin_Created Date', 'weekday_cos_Created Date',
       'month_Created Date', 'day_Created Date', 'geo_cluster', '1_feature',
       '2_feature',
       ...
       '759_feature', '760_feature', '761_feature', '762_feature',
       '763_feature', '764_feature', '765_feature', '766_feature',
       '767_feature', '768_feature'],
      dtype='object', length=776)


In [25]:
lgbm_model = lgb.LGBMClassifier(verbose=-1)

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='multiclass',  
    num_class=unique_classes, 
    metric='multi_error' 
)

1


In [26]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Address '
                                                                  'Type',
                                                                  'combined_text'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(metric='multi_error',
                                                                 num_class=1,
                                                                 objective='multiclass',
                                                                 verbose=-1)))

In [27]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.949584
1,precision,0.510987
2,recall,0.463120
3,f1,0.473513
